---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [287]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer

ImportError: cannot import name 'Imputer' from 'sklearn.preprocessing' (C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\__init__.py)

In [224]:
#Insert lat and lon in address embbeded in adresses csv 
add = pd.read_csv('addresses.csv') 
lat = pd.read_csv('latlons.csv') 
geoRef = pd.merge(add,lat, how='left', on='address')
geoRef.set_index(['ticket_id'], inplace=True)
geoRef=geoRef[['lat','lon']]

train = pd.read_csv('train.csv', sep=',', encoding='cp1252', low_memory=False,
                    parse_dates = ['ticket_issued_date', 'hearing_date'], index_col='ticket_id')
train = pd.merge(train, geoRef, how='left', on='ticket_id')

test = pd.read_csv('test.csv', sep=',', encoding='cp1252', low_memory=False,
                    parse_dates = ['ticket_issued_date', 'hearing_date'], index_col='ticket_id')
test = pd.merge(test, geoRef, how='left', on='ticket_id')

In [225]:
#delete columns in both fields
train_remove = ['payment_amount','payment_date','payment_status','balance_due','collection_status','compliance_detail']
train.drop(train_remove, axis=1, inplace=True)

all_remove = ['inspector_name', 'violator_name', 'violation_street_number', 'violation_street_name', 'violation_zip_code',
              'mailing_address_str_number','mailing_address_str_name', 'city', 'zip_code', 'non_us_str_code', 'country', 
              'violation_description', 'grafitti_status', 'ticket_issued_date', 'hearing_date']

train.drop(all_remove, axis=1, inplace=True)
test.drop(all_remove, axis=1, inplace=True)

print('null values in train set\n{}'.format(train.isnull().sum()))
print('-----\nnull values in test set\n{}'.format(test.isnull().sum()))

null values in train set
agency_name            0
state                 93
violation_code         0
disposition            0
fine_amount            1
admin_fee              0
state_fee              0
late_fee               0
discount_amount        0
clean_up_cost          0
judgment_amount        0
compliance         90426
lat                    3
lon                    3
dtype: int64
-----
null values in test set
agency_name          0
state              331
violation_code       0
disposition          0
fine_amount          0
admin_fee            0
state_fee            0
late_fee             0
discount_amount      0
clean_up_cost        0
judgment_amount      0
lat                  5
lon                  5
dtype: int64


In [226]:
#categorical features
#agency_name, state, violation_code
print('agency_name categories\n-------------')
tr = set(train.agency_name.unique())
te = set(test.agency_name.unique())
is_subset = te.issubset(tr)
print('test categories is a subset of train categories:{}'.format(is_subset))
print('')
print(train.agency_name.value_counts())
print('')
print('state categories\n-------------')
tr = set(train.state.unique())
te = set(test.state.unique())
is_subset = te.issubset(tr)
print('test categories is a subset of train categories:{}'.format(is_subset))
print('')
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", 
          "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR", 
          "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
print('Not US states train set')
print(train[~train['state'].isin(states)]['state'].value_counts())
print('')
print('Not US states test set')
print(test[~test['state'].isin(states)]['state'].value_counts())
print('')
print('violation_code\n-------------')
tr = set(train.violation_code.unique())
te = set(test.violation_code.unique())
is_subset = te.issubset(tr)
print('test categories is a subset of train categories:{}'.format(is_subset))
print('Other strategy: Most common frequencies (30)\n-------------')
#check the percentage in train data
cat = 30 
vio_code_freq = train.violation_code.value_counts()
perc_train = vio_code_freq.values[:cat].sum() / vio_code_freq.values.sum() * 100
print('The codes represented is equal to {:.4g}% of total violated codes in train set'.format(perc_train))
vio_code = vio_code_freq.index[:cat]
#check the percentage in test data
vio_code_test = test.violation_code.value_counts()
perc_test = vio_code_test[vio_code_test.index.isin(vio_code)].sum() / vio_code_test.values.sum() * 100
print('The codes represented is equal to {:.4g}% of total violated codes in test set'.format(perc_test))
#categories unlabeled
print('\nThe 5 unlabeled categories with more frequencies in test set')
print(vio_code_test[~vio_code_test.index.isin(vio_code)][:5])#.index[:7])
train['violation_code_cat'] = [c if c in vio_code else 'other' for c in train.violation_code]
test['violation_code_cat'] = [c if c in vio_code else 'other' for c in test.violation_code]

agency_name categories
-------------
test categories is a subset of train categories:True

Buildings, Safety Engineering & Env Department    157784
Department of Public Works                         74717
Health Department                                   8903
Detroit Police Department                           8900
Neighborhood City Halls                                2
Name: agency_name, dtype: int64

state categories
-------------
test categories is a subset of train categories:True

Not US states train set
NB    77
UK    77
ON    51
BC    17
VI     6
QC     5
PR     5
QL     3
BL     2
Name: state, dtype: int64

Not US states test set
UK    142
ON     53
BC     52
NB     24
VI     23
QC     11
PR      4
QL      1
Name: state, dtype: int64

violation_code
-------------
test categories is a subset of train categories:False
Other strategy: Most common frequencies (30)
-------------
The codes represented is equal to 98.02% of total violated codes in train set
The codes represented is

In [227]:
#disposition and numeric features
#disposition
print('disposition categories\n-------------')
tr = set(train.disposition.unique())
te = set(test.disposition.unique())
is_subset = te.issubset(tr)
print('test categories is a subset of train categories:{}'.format(is_subset))
print('')
print('Check the categories in train set')
print(train.disposition.value_counts())
print('')
print('Check the categories in test set')
print(test.disposition.value_counts())
print('')
#judgment_amount is equal to fine_amount + admin_fee + state_fee + late_fee + clean_up_cost
train['Total_amount'] = train['fine_amount']+ train['admin_fee']+train['state_fee']+train['late_fee']+train['clean_up_cost']
test['Total_amount'] = test['fine_amount']+ test['admin_fee']+test['state_fee']+test['late_fee']+test['clean_up_cost']
#amount is equal or the difference is most 1
train['case'] = np.where((train['Total_amount']== train['judgment_amount']) | (abs(train['Total_amount']-train['judgment_amount'])<1), int(1), int(0))
test['case'] = np.where((test['Total_amount']==test['judgment_amount']) | (abs(test['Total_amount']-test['judgment_amount'])<1), int(1), int(0))
print('check the cases when judgment_amount is different to sum of fine_amount + admin_fee + state_fee + late_fee + clean_up_cost')
print('case is 0 when the amount is different and 1 when is almost equal') 
print('---------------Train set---------------')
print(train.case.value_counts())
print('')
print(train.groupby(['case','disposition'])['violation_code'].count())
print('---------------Test set---------------')
print(test.case.value_counts())
print('')
print(test.groupby(['case','disposition'])['violation_code'].count())
print('')
print('number of cases when case != judgment_amount and judgment_amount != 0\n------------Train case------------' )
print(train[(train['case']==0) & (train['judgment_amount']!=0)]['violation_code_cat'].count())
print('number of cases when case != judgment_amount and judgment_amount != 0\n------------Test case------------' )
print(test[(test['case']==0) & (test['judgment_amount']!=0)]['violation_code_cat'].count())
print('')
print('I drop judgment amount, case and Total_amount as features')

disposition categories
-------------
test categories is a subset of train categories:False

Check the categories in train set
Responsible by Default                138340
Not responsible by Dismissal           48695
Not responsible by City Dismissal      34401
Responsible by Admission               13701
Responsible by Determination            7644
Not responsible by Determination        6639
PENDING JUDGMENT                         387
SET-ASIDE (PENDING JUDGMENT)             304
Responsible (Fine Waived) by Deter       195
Name: disposition, dtype: int64

Check the categories in test set
Responsible by Default                51602
Responsible by Admission               4484
Responsible by Determination           4124
Responsible (Fine Waived) by Deter      781
Responsible - Compl/Adj by Default        6
Responsible - Compl/Adj by Determi        2
Responsible by Dismissal                  1
Responsible (Fine Waived) by Admis        1
Name: disposition, dtype: int64

check the cases wh

In [228]:
all_delete = ['violation_code','judgment_amount', 'case', 'Total_amount']
train.drop(all_delete, axis=1, inplace=True)
train = train[train['state']!='BL']
test.drop(all_delete, axis=1, inplace=True)
train.dropna(subset=['compliance'],inplace=True)

In [252]:
#Strategy - fill na values with IterativeImputer
imputer = IterativeImputer()
train2 = pd.get_dummies(train)
tr_col = train2.columns
train2 = imputer.fit_transform(train2)
test2 = pd.get_dummies(test)
te_col = test2.columns
test2= imputer.fit_transform(test2)

print('missing values on train and train2 data')
print(train.isna().sum())
print('')
train2 = pd.DataFrame(train2, columns = tr_col)
print(train2.isnull().values.any())
print(train2.shape, train.shape)
print('')
print('missing values in test and test2 data')
print(test.isna().sum())
print('')
test2 = pd.DataFrame(test2, columns = te_col)
print(test2.isnull().values.any())
print(test2.shape, test.shape)

missing values on train and train2 data
agency_name            0
state                 84
disposition            0
fine_amount            0
admin_fee              0
state_fee              0
late_fee               0
discount_amount        0
clean_up_cost          0
compliance             0
lat                    2
lon                    2
violation_code_cat     0
dtype: int64

False
(159879, 107) (159879, 13)

missing values in test and test2 data
agency_name             0
state                 331
disposition             0
fine_amount             0
admin_fee               0
state_fee               0
late_fee                0
discount_amount         0
clean_up_cost           0
lat                     5
lon                     5
violation_code_cat      0
dtype: int64

False
(61001, 108) (61001, 12)


In [257]:
y = train2.compliance.astype('int')
feat = list(tr_col)
feat.remove('compliance')
X = train2[feat]
Xval = test2

In [282]:
#Libraries to test data
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler

from sklearn.svm import SVC


In [259]:
#split in two parameters
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, stratify=y)

In [270]:
#dummy classifier
dummy_clf = DummyClassifier(strategy = 'most_frequent').fit(Xtrain,ytrain)
ytrpred = dummy_clf.predict(Xtrain)
ytepred = dummy_clf.predict(Xtest)
print('Train, test and AUCROC for dummy Classifier')
dummy_clf.score(Xtrain,ytrain), dummy_clf.score(Xtest,ytest), roc_auc_score(ytrain, ytrpred), roc_auc_score(ytest, ytepred)

Train, test and AUCROC for dummy Classifier


(0.9274616584243051, 0.9274706029522142, 0.5, 0.5)

In [272]:
#Logistic regression
m1 = LogisticRegression(max_iter = 10000).fit(Xtrain,ytrain)
ytrpred = m1.predict(Xtrain)
ytepred = m1.predict(Xtest)
print('Train, test and AUCROC for logistic regression')
m1.score(Xtrain,ytrain), m1.score(Xtest,ytest), roc_auc_score(ytrain, ytrpred), roc_auc_score(ytest, ytepred)

Train, test and AUCROC for logistic regression


(0.9351091244193513, 0.934676007005254, 0.5628870634607376, 0.5612782521339281)

In [273]:
#decision trees
grid_values = {'max_features': [5,10,15,20,25,30], 'max_depth': [4,6,8,10,15]} 
dtc = DecisionTreeClassifier()
grid_dtc = GridSearchCV(dtc, param_grid = grid_values, scoring='roc_auc')
grid_dtc.fit(Xtrain, ytrain)
print('Grid best parameter: ', grid_dtc.best_params_)
ytrpred = grid_dtc.predict(Xtrain)
ytepred = grid_dtc.predict(Xtest)
print('Train, test and AUCROC for Decision tree')
grid_dtc.score(Xtrain,ytrain), grid_dtc.score(Xtest,ytest), roc_auc_score(ytrain, ytrpred), roc_auc_score(ytest, ytepred)

Grid best parameter:  {'max_depth': 10, 'max_features': 30}
Train, test and AUCROC for Decision tree


(0.8156115913463147,
 0.7950767287135881,
 0.6331706717252361,
 0.6218510997267869)

In [280]:
#random forests
#n_estimators, number of bootstrap samples
#max_depth, maximum depth of the tree
model = RandomForestClassifier(n_jobs=2, random_state=0)
grid_values = {'n_estimators':[5,7,8,10], 'max_depth':[5,8,10,15], 'max_features':[10,15,20,25]}
grid_model = GridSearchCV(model, param_grid=grid_values, scoring="roc_auc")
grid_model.fit(Xtrain, ytrain)
print('Grid best parameter: ', grid_model.best_params_)
ytrpred = grid_model.predict(Xtrain)
ytepred = grid_model.predict(Xtest)
print('Train, test and AUCROC for random forests')
grid_model.score(Xtrain,ytrain), grid_model.score(Xtest,ytest), roc_auc_score(ytrain, ytrpred), roc_auc_score(ytest, ytepred)

Grid best parameter:  {'max_depth': 15, 'max_features': 15, 'n_estimators': 10}
Train, test and AUCROC for random forests


(0.8434183124073688,
 0.8106233017575729,
 0.6356662660222473,
 0.6212825953467866)

In [285]:
#neural network
scaler = MinMaxScaler()
Xtrain_scaled = scaler.fit_transform(Xtrain)
Xtest_scaled = scaler.transform(Xtest)

nn = MLPClassifier(hidden_layer_sizes = [20,20], random_state = 0, solver='lbfgs', max_iter=10000)
grid_values = {'activation':['relu','logistic'], 'alpha':[0.01,1]}
grid_nn = GridSearchCV(nn, param_grid=grid_values, scoring="roc_auc")
grid_nn.fit(Xtrain_scaled, ytrain)
print('Grid best parameter: ', grid_nn.best_params_)
ytrpred = grid_nn.predict(Xtrain_scaled)
ytepred = grid_nn.predict(Xtest_scaled)
print('Train, test and AUCROC for neural networks')
grid_nn.score(Xtrain_scaled,ytrain), grid_nn.score(Xtest_scaled,ytest), roc_auc_score(ytrain, ytrpred), roc_auc_score(ytest, ytepred)

Grid best parameter:  {'activation': 'logistic', 'alpha': 1}
Train, test and AUCROC for neural networks


(0.8217830330454742,
 0.7980698338119978,
 0.6252118008246755,
 0.6163290892468922)

In [6]:
#impossible to compute, takes a long time.

#grid_values = {'C': [0.1, 1], 'gamma': [0.1, 1.0]}, 'class_weight':[{1:10},{1:20},{1:50}]}
#scaler = StandardScaler()
#Xtrain_scaled = scaler.fit_transform(Xtrain)
#Xtest_scaled = scaler.transform(Xtest)

#svm = SVC(kernel='rbf', random_state =0)
#grid_svm_acc = GridSearchCV(svm, param_grid = grid_values, scoring = 'roc_auc')
#grid_svm_acc.fit(Xtrain_scaled, ytrain)
#y_decision_fn_scores_acc = grid_svm_acc.decision_function(Xtest_scaled) 

#print('Grid best parameter (max. accuracy): ', grid_svm_acc.best_params_)
#print('Grid best score (accuracy): ', grid_svm_acc.best_score_)

##Best SVM model 
##svm = SVC(random_state =0).fit(Xtrain_scaled,ytrain)
##ypred = svm.predict(Xtest_scaled)
##svm.score(Xtest_scaled,ytest), svm.score(Xtrain_scaled,ytrain), roc_auc_score(ytest, ypred)

In [286]:
#test2 = pd.get_dummies(X_test)
#X_predict = ml.predict_proba(test2)

In [ ]:
import pandas as pd
import numpy as np

def blight_model():
    
    # Your code here
    
    return # Your answer here